In [1]:
!pip install sounddevice soundfile
import sounddevice as sd
import soundfile as sf
import numpy as np 

In [1]:
import sounddevice as sd

# Liệt kê toàn bộ thiết bị âm thanh
devices = sd.query_devices()

# In danh sách thiết bị
print("Danh sách thiết bị âm thanh có sẵn:\n")
for i, device in enumerate(devices):
    print(f"{i}: {device}")


Danh sách thiết bị âm thanh có sẵn:

0: {'name': 'Microsoft Sound Mapper - Input', 'index': 0, 'hostapi': 0, 'max_input_channels': 2, 'max_output_channels': 0, 'default_low_input_latency': 0.09, 'default_low_output_latency': 0.09, 'default_high_input_latency': 0.18, 'default_high_output_latency': 0.18, 'default_samplerate': 44100.0}
1: {'name': 'Headset (USB Audio)', 'index': 1, 'hostapi': 0, 'max_input_channels': 1, 'max_output_channels': 0, 'default_low_input_latency': 0.09, 'default_low_output_latency': 0.09, 'default_high_input_latency': 0.18, 'default_high_output_latency': 0.18, 'default_samplerate': 44100.0}
2: {'name': 'Microphone (DroidCam Virtual Au', 'index': 2, 'hostapi': 0, 'max_input_channels': 1, 'max_output_channels': 0, 'default_low_input_latency': 0.09, 'default_low_output_latency': 0.09, 'default_high_input_latency': 0.18, 'default_high_output_latency': 0.18, 'default_samplerate': 44100.0}
3: {'name': 'Microphone (e2eSoft iVCam)', 'index': 3, 'hostapi': 0, 'max_input_

In [75]:
import sounddevice as sd
import soundfile as sf
import os

# Cấu hình ghi âm
device_index = 4  # Chọn Microphone Array (AMD Audio Dev)
duration = 4  # Ghi âm trong 5 giây
fs = 44100  # Tần số lấy mẫu

# Chọn user để ghi âm
user_name = "user_A"  # Thay đổi thành "user_B" nếu ghi âm cho user B
save_dir = f"data/{user_name}"

# Tạo thư mục nếu chưa có
os.makedirs(save_dir, exist_ok=True)

# Tìm số thứ tự file tiếp theo
existing_files = [f for f in os.listdir(save_dir) if f.endswith(".wav")]
file_count = len(existing_files) + 1
output_filename = f"{save_dir}/{user_name}_{file_count}.wav"

# Bắt đầu ghi âm
print(f"Bắt đầu ghi âm cho {user_name}, lưu vào {output_filename}...")
recording = sd.rec(
    int(duration * fs), samplerate=fs, channels=1, dtype="int16", device=device_index
)
sd.wait()  # Chờ ghi âm hoàn tất
print("Ghi âm xong!")

# Lưu file WAV
sf.write(output_filename, recording, fs)
print(f"Đã lưu file: {output_filename}")

Bắt đầu ghi âm cho user_A, lưu vào data/user_A/user_A_20.wav...
Ghi âm xong!
Đã lưu file: data/user_A/user_A_20.wav
